# This is a notebook that runs PFLOTRAN in the backend for the breakthrough curve of a column experiment

In [1]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact_manual
import ipywidgets as wd

from os import system

## New functions 
import jupypft.model as mo
import jupypft.parameter as pm

In [2]:
ConcentrationAtInlet = 1.66E-16 #mol/L
InjectTimeInPoreVol = 1.0

In [3]:
TemplateFile = "./template.in"
ColumnModel = mo.model(templateFile=TemplateFile)

Parameters with a fixed value:

In [4]:
Porosity     = pm.Real(tag="<porosity>", value=0.37,   units="adim")
FlowVelocity = pm.Real(tag="<darcyVel>", value=0.7585, units="cm/h")
ColumnLenght = pm.Real(tag="<colLenght>",value=0.50,   units="m")

L = ColumnLenght.value*100.
print(L)

50.0


Parameters with a fixed value but calculated from other parameters

In [5]:
ElutionTime  = pm.Real(tag="<elutionTime>",\
                        value=3600*L*InjectTimeInPoreVol*Porosity.value/FlowVelocity.value,\
                        units="s")

EndTime      = pm.Real(tag="<endTime>",\
                        value=10.*InjectTimeInPoreVol,\
                        units="d")

CFL = 1.0
nZ = 50
deltaX = L/nZ
TimeStep     = pm.Real(tag="<timeStep>",\
                        value=CFL*deltaX*Porosity.value/FlowVelocity.value,\
                        units="h")

Parameters whose values are set by a widget:

In [6]:
LongDisp       = pm.WithSlider(tag="<longDisp>",units="cm")
RateAttachment = pm.WithSlider(tag="<katt>",units="1/h")
RateDetachment = pm.WithSlider(tag="<kdet>",units="1/h")
RateDecayAqueo = pm.WithSlider(tag="<decayAq>",units="1/h")
RateDecayImmob = pm.WithSlider(tag="<decayIm>",units="1/h")

Have a list of all parameters. This should be the conection between the model and variable classes (to be done). 

In [7]:
listOfParameters = pm.Parameter.list_of_vars()

Plotting stuff

In [8]:
def plotResults(FILE = "./pflotran-obs-0.tec"):

    textBoxDimensionless = \
      "Péclet = $\\dfrac{\\rm advection}{\\rm dispersion} = $" + \
      mo.Peclet(LongDisp.value,L,FlowVelocity.value,asString=True)
    
    ObservationPoint = np.loadtxt(FILE,delimiter=",",skiprows=1)
    Cnorm = ObservationPoint[:,1]/ConcentrationAtInlet
    TimeInPoreVolumes = 24.0 * ObservationPoint[:,0] * FlowVelocity.value/(L*Porosity.value)
  
    Legend=["$\\dfrac{[V_{(aq)}]}{[V_{(aq)}]_0}$"]
    plt.figure(figsize=(12,8),facecolor="white")
    
    ## Plot log-scale
    ax1 = plt.subplot(2,2,1)
    ax1.plot(TimeInPoreVolumes,Cnorm,c="purple",lw=3)
    ax1.set_yscale("symlog",\
      linthresh=1.0E-6,subs=[1,2,3,4,5,6,7,8,9])
    ax1.set_ylim([-1.0E-7,1.15])
    ax1.set_xlim([0,6])
    ax1.set_xlabel("Pore Volume [$-$]",fontsize="large")
    ax1.axvline(x=InjectTimeInPoreVol,ls="dotted",c="gray",lw=1)
    ax1.axhline(y=1.0,ls="dashed",c="teal",lw=1)
    ax1.axhspan(ymin=-1.0E-7,ymax=1.0E-6,facecolor="pink",alpha=0.2)
       
    ## Péclet number
    ax1.text(5.5,5.0E-3,textBoxDimensionless,\
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.25),\
        horizontalalignment='right')
    
    ## Plot linear-scale
    ax2 = plt.subplot(2,2,2)
    ax2.plot(TimeInPoreVolumes,Cnorm,c="purple",lw=3,label=Legend[0])
    ax2.set_ylim([-1.0E-2,1.02])
    ax2.set_xlim([0,6])
    ax2.set_xlabel("Pore Volume [$-$]",fontsize="large")
    ax2.axvline(x=InjectTimeInPoreVol,ls="dotted",c="gray",lw=1)
    ax2.axhline(y=1.0,ls="dashed",c="teal",lw=1)
    ax2.legend(fontsize="large",loc="upper right")
    ax2.text(4.0,0.9,"max(C)/C₀ = {:.2f}%".format(max(Cnorm*100)),\
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.25),\
        horizontalalignment='right')
    
    ## Damköhler II numbers
    Damkohler = [mo.DamkII(RateAttachment.value,\
                            LongDisp.value,\
                            FlowVelocity.value,\
                            L,asString=False),
                mo.DamkII(RateDetachment.value,\
                            LongDisp.value,\
                            FlowVelocity.value,\
                            L,asString=False),
                mo.DamkII(RateDecayAqueo.value,\
                            LongDisp.value,\
                            FlowVelocity.value,\
                            L,asString=False),
                mo.DamkII(RateDecayImmob.value,\
                            LongDisp.value,\
                            FlowVelocity.value,\
                            L,asString=False)]
    tickLabels = ["$k_{\\rm att}$","$k_{\\rm det}$","λ$_{\\rm aq}$","λ$_{\\rm im}$"]
    
    ax3 = plt.subplot(2,2,3)
    ax3.bar(x=range(len(Damkohler)), height=Damkohler, tick_label=tickLabels, log=True, hatch="/")
    ax3.set_ylabel("Damköhler(II) = $\\dfrac{\\rm reaction}{\\rm dispersion}$",fontsize='large')
    
    ## Parameters of the reaction sandbox
    parametersTable = [["$k_{\\rm att}$",RateAttachment.strValue,"h$^{-1}$"],\
                       ["$k_{\\rm det}$",RateDetachment.strValue,"h$^{-1}$"],\
                       ["$\lambda_{\\rm aq}$",RateDecayAqueo.strValue,"h$^{-1}$"],\
                       ["$\lambda_{\\rm im}$",RateDecayImmob.strValue,"h$^{-1}$"],\
                       ["α$_{\\rm L}$",LongDisp.strValue,"cm"]]
    
    ax4 = plt.subplot(2,2,4)
    table = ax4.table(cellText=parametersTable,\
              colLabels=["Parameter","Value","Unit"],\
              loc='center',colWidths=[0.3,0.3,0.3],edges="horizontal")
    table.set_fontsize(34)
    table.scale(1,2.5)
    ax4.axis('off')
    
    plt.tight_layout()  
    plt.show()

Dummy function that activates when the widget Interact button is pressed

In [9]:
def RunAll(logLongDisp,logKAtt,logKDet,logDecayAq,logDecayIm):
    ColumnModel.cloneTemplate()
    
    RateAttachment.value = logKAtt
    RateDetachment.value = logKDet
    RateDecayAqueo.value = logDecayAq
    RateDecayImmob.value = logDecayIm
    LongDisp.value = logLongDisp
       
    for parameter in listOfParameters:
        ColumnModel.replaceTagInFile(parameter)
    
    ColumnModel.runModel()
    ColumnModel.fixTecFile()
    plotResults()

Define the sliders we want to use in the WithSlider parameters. This cell assigns Float sliders to all:

In [10]:
RateAttachment.slider = \
    wd.FloatSlider(value=0.0400, min=0, max=1, step=0.1,\
                      description=r'\(k_{\text{att}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDetachment.slider = \
    wd.FloatSlider(value=0.0026, min=0, max=1, step=0.1,\
                      description=r'\(k_{\text{de}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDecayAqueo.slider = \
    wd.FloatSlider(value=0.0070, min=0, max=1, step=0.1,\
                      description=r'\(\lambda_{\text{aq}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDecayImmob.slider = \
    wd.FloatSlider(value=0.0350, min=0, max=1, step=0.1,\
                      description=r'\(\lambda_{\text{im}}\) [1/h]', \
                      style={'description_width': 'initial'})

This cell below assigns `FloatLog` sliders to all

In [11]:
RateAttachment.slider = \
    wd.FloatLogSlider(value=0.0400,base=10, min=-30, max=1, step=0.1,\
                      description=r'\(k_{\text{att}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDetachment.slider = \
    wd.FloatLogSlider(value=0.0026,base=10, min=-30, max=1, step=0.1,\
                      description=r'\(k_{\text{de}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDecayAqueo.slider = \
    wd.FloatLogSlider(value=0.0070,base=10, min=-30, max=1, step=0.1,\
                      description=r'\(\lambda_{\text{aq}}\) [1/h]', \
                      style={'description_width': 'initial'})

RateDecayImmob.slider = \
    wd.FloatLogSlider(value=0.0350,base=10, min=-30, max=1, step=0.1,\
                      description=r'\(\lambda_{\text{im}}\) [1/h]', \
                      style={'description_width': 'initial'})

LongDisp.slider = \
    wd.FloatLogSlider(value=0.2,base=10, min=-30, max=2, step=0.1,\
                      description=r'\(\alpha_{L}\) [cm]', \
                      style={'description_width': 'initial'})

This cell below assigns Float sliders to only the LongDisp

In [12]:
LongDisp.slider = \
    wd.FloatSlider(value=0.2,min=0.0, max=2, step=0.1,\
                      description=r'\(\alpha_{L}\) [cm]', \
                      style={'description_width': 'initial'})

In [16]:
interact_manual(RunAll,\
    logLongDisp = LongDisp.slider,\
    logKAtt = RateAttachment.slider,\
    logKDet = RateDetachment.slider,\
    logDecayAq = RateDecayAqueo.slider,\
    logDecayIm = RateDecayImmob.slider\
    );
              

interactive(children=(FloatSlider(value=0.2, description='\\(\\alpha_{L}\\) [cm]', max=2.0, style=SliderStyle(…

**Péclet = Advection Rate/Dispersion Rate**

$
    \text{P}_{é} = \dfrac{LU}{D} = \dfrac{LU}{\alpha_L U} = \dfrac{L}{\alpha_L}
$

**Damköhler(II) = Reaction Rate/Dispersion Rate**

$
    \text{D}_{A,II} = \dfrac{L^2k}{D} = \dfrac{L^2k}{\alpha_LU}
$

      "Damköhler(II) = $\\dfrac{\\rm reaction}{\\rm dispersion}$"+"\n" +\
      "Da$^{\\rm kat} = $"+ PFLO.DamkII(RateAttachment.get_value(),\
                                     LongDisp.get_value(),\
                                     FlowVelocity.get_value(),\
                                     L,asString=True) +"\n" +\
      "Da$^{\\rm kde} = $"+ PFLO.DamkII(RateDetachment.get_value(),\
                                     LongDisp.get_value(),\
                                     FlowVelocity.get_value(),\
                                     L,asString=True) +"\n" +\
      "Da$^{\\rm λaq} = $"+ PFLO.DamkII(RateDecayAqueo.get_value(),\
                                     LongDisp.get_value(),\
                                     FlowVelocity.get_value(),\
                                     L,asString=True) +"\n" +\
      "Da$^{\\rm λim} = $"+ PFLO.DamkII(RateDecayImmob.get_value(),\
                                     LongDisp.get_value(),\
                                     FlowVelocity.get_value(),\
                                     L,asString=True) +"\n" +\
                                     
    
    ## Rate values
    ax1.text(5.5,5.0E-4,textBoxKin,\
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),\
        horizontalalignment='right')                           
                                     
    textBoxKin = \
      "$k_{\\rm att} = $" + RateAttachment.get_strValue() + " h$^{-1}$" +"\n" + \
      "$k_{\\rm det} = $"+ RateDetachment.get_strValue() + " h$^{-1}$" +"\n" + \
      "$\lambda_{\\rm aq} = $ "+ RateDecayAqueo.get_strValue() + " h$^{-1}$" +"\n" + \
      "$\lambda_{\\rm im} = $ "+ RateDecayImmob.get_strValue() + " h$^{-1}$" +"\n" + \
      "α$_{\\rm L} = $ " + LongDisp.get_strValue() + " cm "